<a href="https://colab.research.google.com/github/Kyveli-tsioli/hello-world/blob/main/Online_Lecture_10_01_Recurrent_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'

--2020-11-05 07:54:17--  https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to julialang-s3.julialang.org (julialang-s3.julialang.org)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 302 gce internal redirect trigger
Location: https://storage.googleapis.com/julialang2/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz [following]
--2020-11-05 07:54:17--  https://storage.googleapis.com/julialang2/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.253.63.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88706549 (85M) [application/octet-stream]
Saving to: ‘/tmp/julia.tar.gz’

/tmp/julia.tar.gz   100%[

In [ ]:
using Pkg

pkg"add Plots"

pkg"precompile"

using Plots



#this is our first advanced architecture
#a sequence of non linearities and standard matrix transformations
#mlp are still primitive architectures in neural networks in general
#fixed sized inputs (so far), we mean each input has the exact same dimensionality (assignm1: 28x28 all the inputs)
#this is not always the case (images can be of different sizes)
#variable length inputs
#recurrent neural network: some sort of differentiable state machine
# tracks some sort of previous state (h)
# doesnt come due to our current input, it comes from the previous input at time state h(t-1)
#x(t) input at t
#produces an output h(t)
#x(t) goes to a processing box it produces an h(t): a reccurent connection
#each hidden state lies on a d -space
# weight matrix lies dx2d (maps tow of these vectors into 2d vectors)
#concatenation between previous hidden state and current input that go to our weight matrix, we apply a non linearity (tanh) and we produce an output
#dim of h is a hyperparameter
#x's tend to be either embeddings or 1-hot vectors 
#time series and nlp, reinforcement learning: recurrent neural networks
#keeps track of a world state: predicting an action, updating the world, keep track of an agent
#biomedical research, sequences of symbols
#h(t):once we have an input x(t+1) we go to x(t)
#which time step i am currently in
#see it as we take simply take multiple copies of the same value 
#h0 goes as an output and passed thought A
#each time steps (rectangular with the 2 circles) will use the same W (weight tighing)
#the previous h passed on the next layer (A)
#the first layer not getting an h (h(-1)) commonly its a vector of dimension 0 with zeros
#mlp: a single layer takes an input of some dimensionality and produces an output of some other dimensionality (there is no right or wrong here, it is a transformation from which we can learn the weigthts, computational unit)
#tanh: activation function
#outputs are the same dimension as the input
#W is 2D 
# domain of the output: tanh bounded between -1 and 1 (not as logistic function)
#largest gradient at the origin (0) as the logistic
#when you go far from the origin you have no gradient at all
#ht=tanh(W[h(t-1),x(t)]): what kind of behavior we loose if we put logistic
#you cant produce negatives (a state being reverted)
#in temrs of what can be expressed on a hidden state
#you are always bounded 
#you can produce negative outputs
#problem of exploding gradients: exploding outputs 
#the logistic function has output domain [0,1] no matter weights and x you have, you can never have a negative output
#vanilla tanh1: loss in terms of numerical precision
#you can scale the logistic outputs by 2x-1 would that give the same as tanh (transformation)
#vanishing gradients: key problem with rnns
#phenomenon where due to non linearities and initialisations you got up woth vanishing gradients
#input x0: you want to predict a grammatially dependend word on x0
#you take the gradient
#error term we respect to our state h3
#push the error term down (backpropagation)
#non linearity used to produce h0, h1, h2, h3
#when you want to push the gradient back through
#the gradient becomes weaker and weaker ("backpropagation through time")
#activation function has multiple effects
#turing completeness: able to compute any computable function
#you can untie the weights, you have more weights to learn if you do this
#tanh: has a larger gradient and that helps (when comparing with logistic)
#h is a bit like a markov process except with activation funtions between every step
#you can use these models in a similar way to hidden markov models#no loss/gradient from intermediate h
#vanishing gradients problem: 



# Recurrent neural networks #

It is time for our first “advanced” architecture!

## End of intermission: The problem of variable length inputs ##

So far all of the inputs to our neural networks have been “fixed size”, in that each $x$ has had the same dimensionality. But this is not always the case:

* Language: Not all sentences have the same length
* Time series: Varying history length

There is also the issue of images of different sizes, but let us ignore that for now…

## The “vanilla” Recurrent Neural Network ##

Conceptually we can think of a Recurrent Neural Network (RNN) as a state machine that takes its previous output $h_{t - 1}$, an input at a given time $t$, $x_t$, and produces an output $h_t$. Early in the literature they were illustrated precisely like a state machine, see the illustration below.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-rolled.png)

[Image source](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

$$ x_i \in \mathbb{R}^{d} $$

$$ h_i \in \mathbb{R}^{d} $$

$$ W \in \mathbb{R}^{d \times 2d} $$

$$ h_t = \textrm{tanh}(W[h_{t - 1}, x_t]) $$

However, these while these illustrations are simple and compact, it is easier to “unroll” an RNN when discussing its behaviour. Exactly as we can see below.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

[Image source](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

This is a very powerful model, even proven to be Turing complete.

### Activation function reminder ###

Just to refresh our memory, here are the two activations functions we will use throughout today’s lecture.

In [ ]:
tanhprime(x)     = 1 .- tanh(x).^2
plt = plot(tanh, xlimits=(-5, 5), label="tanh", legend=:bottomright)
plt = plot!(plt, tanhprime, label="tanh'")
logistic(x)      = 1/(1 .+ exp.(-x))
logisticprime(x) = logistic(x).*(1 .- logistic(x))
plt = plot!(plt, logistic, label="logistic")
plot!(plt, logisticprime, label="logistic'")





#vanilla rnn: x(t)combined using a weight matrix w, apply activation function which produces a new h state
#we learn the weights: it depends on what function we use (logistic function/regression etc)
#rtsm
#cell state (c) or long term memory: the state that we pass 
#gates: a forget gate is a set of weights wf f for forgeting, we apply it to the prrevious h(t-1) conatenate it with the x(t -the input, you add the bias, each gate has a unique bias and weight)
#the output will be bounded between 0 and 1 because we used the sigmoid
#elemnt wise multiplication with the previous state (h(t-1))
#forget it (0) or maintain it (1) or scale it if between 0 and 1
#input scaling factor i(t): Wi our input weights, we take our previous output, concatenate with current input x(t) add a bias, linear transformation and then logistic function
#cts candidate activations
#the bias can be added in the weight matrix
#sell state update: what it does?
#Ct is a vector
#element wise calculations all of these 
#forget gate: d dimensional vector f(t) same dimensionality wth C(t-1):scale each and every component
#it is a scaling (0,1) it comes from the logistic
#output gate: another weight matrix 
#o(t): different weight matrix now
#gain scaled between 0 and 1
#linealry scale based on the output gate (h(t)=o(t)*tanh(Ct))
#ht is our output that we pass in the next state
#the regularisation of information at each step help with vanishing gradients 
#vanilla rnn: for each time step you project your entire state and scale your entire state before pass into the next step
#dont have any weight matrix that directly with cell state 
#vanishing gradient: you want to have a gradient with respect to the input and this gradient needs to be backpropagated
#output gate: we produce the output ot 
#ht is conditional on c
#wf controls the amount of information how much you are supposed to forget inide the cell 
#ft is vector to apply to the previous 
#ft can be 1 or 0 : either preserves what is in the current cell or scales it
#shared for each time step the weight matrices
#activation function: logistic function
#the use of 0_t is just to calculate h_t (ot to scale each component of the output)
#turing completeness proof for the rnn
#how complex behavior ltsm can learn?
#Kapathy: softmax predicted the next character (blue red text in the notebook)
#off diagonal strong interactions, in the diagonal the info is retained
#vp-np you preserve information 
#bi derictional rnn : 1 rnn from left to right and 1 from right to left and then concatenate the output states in order to produce predictions
#you carry out backpropagation as if the unrolled weights are all separate, and then you just add up all of those gradients and update the weights with that sum of gradients
#gradient exploding can happen it depends on the initialisation
#if you want to capture some of the long term relationships in language, then vanishing gradients is a problem
#we initialise to 1 because we dont want to forget at first
#tensorflow initialises at zero (probably) bf=0(no contribution at all, no bias towards initial forgetting)
#bf=1 you end up remembering most of the cell states
#we cant initialise all of our weight matrices using small value
#bf=1 rather than 0
#all the functions are element wise 


### Exploding gradient ###

We note the $\textrm{tanh}$ activation function being used for our “vanilla” RNN. Why is it necessary? Given the term “exploding gradient” what do you believe it signifies? When it is a problem?

### Vanishing gradient ###

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)

Imagine that an output $h_3$ is to be influenced by observations $x_0$ and $x_1$, this is exactly the kind of behaviour we expect an RNN to be able to learn.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)

However, by virtue of the inner workings of the “vanilla” RNN. As the distance between an output and its related inputs increase, it becomes more difficult to learn the desired behaviour, sometimes even impossible. *Why?*

### Long Short-Term Memory ###

The problem of vanishing gradients plagued the field througouht the 90s, but unlike the vanishing gradient for multi-layer perceptrons which we only came to terms with in the late 00s, [Hochreiter and Schmidhuber (1997)](https://www.bioinf.jku.at/publications/older/2604.pdf) proposed a fairly ingenious solution to the problem – Long Short-Term Memory, commonly referred to as an LSTM.

Consider our “vanilla” RNN.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)

$$ h_t = \textrm{tanh}(W[h_{t - 1}, x_t]) $$

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

### The “cell state”, our long-term memory ###

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png)

### Forget gate ###

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png)

### Input gate ###

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png)

### Cell state update ###

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png)

### Output gate ###

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png)

What are some positive downsides of the LSTM? Does it still suffer from exploding/vanishing gradients or is it immue to these phenomena under all circumstances? In 2016 (or was it 2015?) Google Research published a paper on a minute detail regarding the initialisation of the LSTM weights at ICML. Allegedly Schmidthuber upon seeing the work as a poster presentation commented to the authors that “Oh, yes, we forgot to mention that, didn’t we?”. What do you think this detail was?


### LSTM variants ###

There is no standard formulation of the LSTM, although what we used above is the most dominant. Here are two alternative formulations. Can we derive their motivations from their equations?

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-peepholes.png)

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-tied.png)

In practice the “simpler” formulation is “powerful enough” and you rarely see these variants.

#### Gated Recurrent Unit (GRU) ####

Introduced by Cho et al. (2014) as a more computationally efficient variant to the LSTM. Again, once you are familiar with the LSTM it becomes fairly straight forward to analyse its behaviour from its equations.

![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png)

### So, what does an LSTM learn? ###

In theory, even a “vanilla” RNN is Turing complete ([Siegelmann and Sontag, 1995](http://research.cs.queensu.ca/home/akl/cisc879/papers/SELECTED_PAPERS_FROM_VARIOUS_SOURCES/05070215382317071.pdf)). But at this point, we can draw parallels to how we know that while neural networks are universal function approximators, we do know that ultimately we are limited by what we can learn given finite amounts of data and compute. So, how complex of a behaviour can these models learn?

![](https://d3i71xaburhd42.cloudfront.net/40be3888daa5c2e5af4d36ae22f690bcc8caf600/5-Figure2-1.png)

[Image source](https://www.semanticscholar.org/paper/Visualizing-and-Understanding-Recurrent-Networks-Karpathy-Johnson/40be3888daa5c2e5af4d36ae22f690bcc8caf600)

From [“Visualizing and Understanding Recurrent Networks”](https://arxiv.org/abs/1506.02078) by Karpathy et al. (2015), which is a remarkably accessible paper.

## Structure-guided recurrent neural networks ##

For a brief moment in time, referred to as “recursive neural networks”.

![](https://d3i71xaburhd42.cloudfront.net/0c23ebb3abf584fa5e0fde558584befc94fb5ea2/1-Figure1-1.png)

[Image source](https://www.semanticscholar.org/paper/Parsing-Natural-Scenes-and-Natural-Language-with-Socher-Lin/9c0ddf74f87d154db88d79c640578c1610451eec)

### Sentiment analysis ##

![](https://d3i71xaburhd42.cloudfront.net/961f6d5bb9b43833d7387e377b25d7a68118b23b/3-Figure2-1.png)

[Image source](https://www.semanticscholar.org/paper/Recursive-Deep-Models-for-Semantic-Compositionality-Socher-Perelygin/687bac2d3320083eb4530bf18bb8f8f721477600)

More examples at the [Stanford Sentiment Treebank homepage](https://nlp.stanford.edu/sentiment/treebank.html).

### Syntactic analysis  ###

![](https://upload.wikimedia.org/wikipedia/commons/5/54/Parse_tree_1.jpg)

[Image source](https://en.wikipedia.org/wiki/File:Parse_tree_1.jpg)

![](https://d3i71xaburhd42.cloudfront.net/acc4e56c44771ebf69302a06af51498aeb0a6ac8/1-Figure1-1.png)

![](https://d3i71xaburhd42.cloudfront.net/acc4e56c44771ebf69302a06af51498aeb0a6ac8/9-Figure5-1.png)

[Image source](https://www.semanticscholar.org/paper/Parsing-with-Compositional-Vector-Grammars-Socher-Bauer/acc4e56c44771ebf69302a06af51498aeb0a6ac8)


### 2014 to 2018: RNN variants ###

For a few years, RNNs with LSTM cells dominated the field of natural language processing. Below are two very common examples.

#### Sequence to Sequence RNN ####

![](https://d3i71xaburhd42.cloudfront.net/cd52da21cdec50b25b6fb0ba6741091ad38fc986/2-Figure1-1.png)

#### Bi-directional RNN ####

![](https://d3i71xaburhd42.cloudfront.net/8dd6aae51e31a72752c4be5cddbdd76dfdc6cda4/4-Figure3-1.png)

## Additional reading ##

1. [“The Unreasonable Effectiveness of Recurrent Neural Networks”](https://karpathy.github.io/2015/05/21/rnn-effectiveness)
2. [“The Unreasonable Syntactic Expressivity of RNNs”](https://nlp.stanford.edu/~johnhew/rnns-hierarchy.html)

## Acknowledgements ##

A big thank you to Christopher Olah for his [“Understanding LSTM Networks”](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) (2015) from which I myself and many others “steal” the excellent illustrations.